<a href="https://colab.research.google.com/github/zyf2021/-/blob/main/HW_Tinkoff_Pandas_Unit_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Предполагается, что в рамках данного задания ученики сами будут изучать документацию, как минимум используя функционал `tab` и `shift+tab` от Jupyter

1. Импортируйте библиотеку pandas 
2. Считайте файл `wage.csv` в DataFrame с названием `wage`. Используйте для этого функцию pd.read_csv()

`person_id` - идентификатор человека

`gender` - пол, 0 - женский, 1 - мужской

`wage` - заработная плата в рублях

In [2]:
import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/content/drive')

wage = pd.read_csv('/content/drive/MyDrive/wage-1-ee13a6b1-605e-44fe-83a6-85d3d7a0b3ae.csv')
wage

Mounted at /content/drive


,person_id,gender,wage
0,0,1,46793.603811
1,1,1,33481.575720
2,2,1,44523.699084
3,3,1,15995.576829
4,4,0,10282.631224
...,...,...,...
1001,995,1,66503.737185
1002,996,1,9972.956272
1003,997,0,104504.616392
1004,998,1,98927.903076


3. Поменяйте колонку `gender` так, чтобы были записи `F` (female) и `M` (male) вместо 0 и 1

In [3]:
wage.loc[(wage.gender==0),('gender')] = 'F'
wage.loc[(wage.gender==1),('gender')] = 'M'

In [ ]:
wage

,person_id,gender,wage
0,0,M,46793.603811
1,1,M,33481.575720
2,2,M,44523.699084
3,3,M,15995.576829
4,4,F,10282.631224
...,...,...,...
1001,995,M,66503.737185
1002,996,M,9972.956272
1003,997,F,104504.616392
1004,998,M,98927.903076


4. Давайте посчитаем среднюю зарплату у мужчин и женщин. Для этого используйте метод `groupby` со следующим синтаксисом:

`dataframe.groupby(Название_колонки_для_группировки)[Перечисление_колонок_для_агрегации].функция_агрегации()`

In [4]:
wage.groupby('gender')['wage'].mean()

gender
F    40855.747261
M    46815.944005
Name: wage, dtype: float64

5. Теперь взглянем внимательнее на данные и обнаружим, что некоторые люди попали в выборку несколько раз. 
    1. Найдите таких людей. Подсказка: `value_counts()`
    0. Убедитесь, что записи по ним с одинаковым `wage`. Возможно, тут вам пригодится функция агрегации `nunique()`, отображающая количество разных значений
    0. Избавьтесь от повторяющихся значений. Подсказка: `drop_duplicates()`

In [5]:
#wage.groupby('person_id')['person_id'].value_counts()
p_i = wage['person_id'].value_counts().loc[lambda x : x > 1]
p = np.array(p_i.index)

In [6]:
filter = wage['person_id'].isin(p)
wage[filter].nunique()

person_id    6
gender       2
wage         6
dtype: int64

In [7]:
wage[filter].value_counts()

person_id  gender  wage         
12         M       23547.663281     2
13         M       273483.609236    2
14         M       13434.655493     2
15         F       68341.332163     2
16         F       35448.040878     2
17         M       20178.985060     2
dtype: int64

In [8]:
wage.drop_duplicates(['person_id'],keep='first',inplace=True)
print(wage.person_id.nunique())

1000


In [ ]:
person_id = wage.person_id.value_counts()
df_person = pd.DataFrame(person_id)
df_person = df_person.reset_index()
df_person.columns = ['person_id', 'number']
df_person.loc[(df_person.number>1),'number']
#wage.wage.nunique()
#wage.loc[(wage.person_id.value_counts())>2, :]


0    2
1    2
2    2
3    2
4    2
5    2
Name: number, dtype: int64

6. Теперь посмотрим внимательнее на зарплаты
    1. Охарактеризуйте имеющиеся данные по зарплатам. Подсказка: `describe`
    1. Избавьтесь от бессмысленных значений

In [9]:


wage = wage[wage['wage']>0]

wage

print(wage.wage.describe())

count       995.000000
mean      44306.969585
std       54302.194392
min         945.648458
25%       14683.306148
50%       27519.361794
75%       52267.313664
max      755320.874132
Name: wage, dtype: float64


7. Давайте теперь посмотрим на зарплату с учетом бонуса. Для этого нам понадобится таблица `bonus.csv`. Считайте ее в переменную `bonus`. Заметьте, что она сохранена немного в другом формате, и вам понадобится уточнить параметр `sep` - разделитель записей. Сравните текущий файл с предыдущим и попробуйте решить проблему

In [10]:
bonus = pd.read_csv('/content/drive/MyDrive/bonus-1-c177edc7-6e8e-4e53-8f5d-098bd95cc4de.csv', sep = ';')

In [11]:
bonus.head()

,person_id,bonus
0,905,85059.638382
1,836,7703.346074
2,287,3120.269742
3,548,5347.987142
4,575,137257.490614


8. Чтобы посчитать итоговую зарплату, нам нужно по каждому человеку знать и оклад, и премию. Для этого надо будет соединить (сджойнить) таблицы по `person_id`. Используйте для этого функцию `pd.merge`. Помните, что параметр `how` должен быть `'outer'`, чтобы сохранить те записи, что есть только в одной таблице. Результат запишите в новый dataframe `df`

In [12]:
df = pd.merge(wage, bonus, 'outer', 'person_id')
df

,person_id,gender,wage,bonus
0,0,M,46793.603811,3.332934e+04
1,1,M,33481.575720,NaN
2,2,M,44523.699084,3.192912e+06
3,3,M,15995.576829,2.196858e+04
4,4,F,10282.631224,NaN
...,...,...,...,...
994,997,F,104504.616392,5.380978e+04
995,998,M,98927.903076,NaN
996,999,M,108107.141368,NaN
997,43,NaN,NaN,2.687784e+05


In [13]:
df.isnull().any(axis=0)
df['bonus'] = df['bonus'].fillna(0)
df = df.dropna()
df.isnull().any(axis=0)
#df['total'] = df['wage']*12 + df['bonus']
df.loc[:, 'total'] = df.wage * 12 + df.bonus
df


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,person_id,gender,wage,bonus,total
0,0,M,46793.603811,3.332934e+04,5.948526e+05
1,1,M,33481.575720,0.000000e+00,4.017789e+05
2,2,M,44523.699084,3.192912e+06,3.727197e+06
3,3,M,15995.576829,2.196858e+04,2.139155e+05
4,4,F,10282.631224,0.000000e+00,1.233916e+05
...,...,...,...,...,...
992,995,M,66503.737185,3.452137e+03,8.014970e+05
993,996,M,9972.956272,3.892790e+05,5.089544e+05
994,997,F,104504.616392,5.380978e+04,1.307865e+06
995,998,M,98927.903076,0.000000e+00,1.187135e+06


In [14]:
#df.groupby('gender')['total'].agg(['mean','median'])
df.groupby('gender').wage.agg(['median', 'mean'])

,median,mean
gender,,
F,26721.001610,41353.986687
M,29006.633828,47081.694269


9. Наконец, давайте посчитаем итоговую зарплату
    1. Замените отсутствующие записи в колонке `bonus` нулями
    1. Уберите людей без `wage` - это те "плохие" записи, от которых мы избавлялись на предыдущих шагах
    1. Сделайте новую колонку `total`, которая будет равна 12 окладам и премии
    1. Посчитайте среднюю и медианную итоговую зарплату в разрезе по полу. Подсказка: вместо функции агрегации можно написать `.agg()` и перечислить внутри нужные агрегаты

10. Сохраните `df` в файл, используя метод `to_csv()`. Не записывайте индексы

In [ ]:
df.to_csv(path_or_buf='tinkoff_csv_hw',index=False)